# API Geo Reverse Toronto Home Sales

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

In [2]:
home_sales_df = pd.read_csv("Resources/toronto_home_prices_Sayeed.csv")
home_sales_df.head()

,Index,AreaName,Price ($),lat,lng,Address,City
0,3901,East Danforth,49900,43.682693,-79.325851,1475 Danforth Ave,Toronto
1,3902,Downtown,379880,43.647514,-79.392700,#2705 - 101 PETER ST,Toronto
2,3903,Woburn,419900,43.773827,-79.250023,#2308 - 88 Grangeway Ave,Toronto
3,3904,The Junction,49900,43.669209,-79.481644,2411 St Clair Ave,Toronto
4,3905,Cliffcrest,4500000,43.725761,-79.218529,25 Meadowcliffe Dr,Toronto


In [3]:
toronto_home_sales = home_sales_df.drop(columns="Index")
toronto_home_sales["Postal Code"] = ""
toronto_home_sales.head()

,AreaName,Price ($),lat,lng,Address,City,Postal Code
0,East Danforth,49900,43.682693,-79.325851,1475 Danforth Ave,Toronto,
1,Downtown,379880,43.647514,-79.392700,#2705 - 101 PETER ST,Toronto,
2,Woburn,419900,43.773827,-79.250023,#2308 - 88 Grangeway Ave,Toronto,
3,The Junction,49900,43.669209,-79.481644,2411 St Clair Ave,Toronto,
4,Cliffcrest,4500000,43.725761,-79.218529,25 Meadowcliffe Dr,Toronto,


In [4]:
# params dictionary to update each iteration
params = {"key": gkey}

# Use the lat/lng we recovered to identify postal code
for index, row in toronto_home_sales.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["latlng"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

    # make request
    postal_code = requests.get(base_url, params=params)

    # convert to json
    postal_code = postal_code.json()
    
    try:
        toronto_home_sales.loc[index, "Postal Code"] = postal_code["results"][0]["address_components"][7]["long_name"]
    except (IndexError):
        print(f"No Results Found: {lat}, {lng}")

No Results Found: 43.6479, -79.38203399999999
No Results Found: 43.596352, -79.583061
No Results Found: 43.851192, -79.25350999999999
No Results Found: 43.654224, -79.353928
No Results Found: 43.648635999999996, -79.381744
No Results Found: 43.752876, -79.624969
No Results Found: 43.748321999999995, -79.34655
No Results Found: 43.684253999999996, -79.346863
No Results Found: 43.716797, -79.259888
No Results Found: 43.754265000000004, -79.468536
No Results Found: 43.659836, -79.384956
No Results Found: 43.78833, -79.264862
No Results Found: 43.754096999999994, -79.619812
No Results Found: 43.651909, -79.569687
No Results Found: 43.651169, -79.384987
No Results Found: 43.720496999999995, -79.456047
No Results Found: 43.654728000000006, -79.398453
No Results Found: 43.747440000000005, -79.311684
No Results Found: 43.777565, -79.281006
No Results Found: 43.708683, -79.47789
No Results Found: 43.716564, -79.261368
No Results Found: 43.820499, -79.295738
No Results Found: 43.654728000000006,

In [5]:
# Visualize to confirm airport data appears
toronto_home_sales.head(10)

,AreaName,Price ($),lat,lng,Address,City,Postal Code
0,East Danforth,49900,43.682693,-79.325851,1475 Danforth Ave,Toronto,M4J 1N5
1,Downtown,379880,43.647514,-79.392700,#2705 - 101 PETER ST,Toronto,M5V 0G6
2,Woburn,419900,43.773827,-79.250023,#2308 - 88 Grangeway Ave,Toronto,M1H 0A2
3,The Junction,49900,43.669209,-79.481644,2411 St Clair Ave,Toronto,M6N 1K9
4,Cliffcrest,4500000,43.725761,-79.218529,25 Meadowcliffe Dr,Toronto,M1M 2X9
5,Downtown,99000,43.647900,-79.382034,#58 - 77 KING ST W,Toronto,
6,Lansing,660000,43.760517,-79.411743,#2002 - 11 BOGERT AVE,Toronto,M2N 0H3
7,Mimico,369000,43.620010,-79.483086,#104 - 2261 Lake Shore Blvd,Toronto,M8V 3X1
8,Bathurst Manor,780000,43.753391,-79.448715,#Th2 - 784 Sheppard Ave,Toronto,M3H 2T1
9,The Queensway,549900,43.624573,-79.511681,#202 - 51 Lady Bank Rd,Toronto,M8Z 1N7


In [6]:
# Save Data to csv
toronto_home_sales.to_csv("output_data/toronto_api_postal_sh.csv")